In [1]:
import pandas as pd

In [5]:
# read in the file
df = pd.read_csv('PD 2020 WK 1 Input - Sheet1.csv')
df

,Item,Profit
0,1. Bar Soap,NaN
1,1.1 Opaque,NaN
2,1.1.1 Lavender,10778.0
3,1.1.2 Honey,-6103.0
4,1.1.3 Orange,5422.0
5,1.1.4 Mint,6301.0
6,1.2 Transparent,NaN
7,1.2.1 Lavender,10769.0
8,1.2.2 Honey,-270.0
9,1.2.3 Orange,-4981.0


In [4]:
# replace nulls with zeroes
df['Profit'].fillna(0, inplace=True)

In [6]:
# extract the hierarchy from the Item and copy it
df['Hierarchy'] = df['Item'].str.extract('([\d\.]+?)\.? .*')
df['Hierarchy2'] = df['Hierarchy']

In [7]:
# create a level column to help with the looping in the next step
df['Level'] = df['Hierarchy'].str.count('\.')
maxLevel = df['Level'].max()

In [11]:
# iterate through the hierarchy levels, creating subtotals at each level
dfSubtotal = None
for i in range(maxLevel, 0, -1):
    # remove a layer of hierarchy. strip off a period(.) and a digit from the end
    df['Hierarchy2'] = df['Hierarchy2'].str.extract('(.*?)\.\d+$')

    # using only the detail records, sum by current level of hierarchy
    # and add to the Subtotal df
    dfSubtotal = pd.concat([dfSubtotal,
                            df[df['Level'] == maxLevel].groupby(df['Hierarchy2'], as_index=True)['Profit'].sum()])

In [13]:
from IPython.display import Video

Video("2020 w1 loop.mp4")

In [14]:
# join subtotals back to the main dataframe and update the Profit
df = pd.merge(df, dfSubtotal, how='left', left_on='Hierarchy', right_index=True)
df['Profit'] = df['Profit_x'] + df['Profit_y'].fillna(0)

In [16]:
# add the spacing to the Item
df['Item'] = [' ' * 5 * x for x in df['Level']] + df['Item']

In [17]:
# clean up columns
df = df[['Item', 'Profit']]